In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from subprocess import check_output
print (check_output(["ls","./"]).decode("utf-8"))

Deeplearning.ipynb
Sign-Language-Digits-Dataset
Signlanguage.ipynb



In [9]:
x_l= np.load('./Sign-Language-Digits-Dataset/X.npy')
y_l= np.load('./Sign-Language-Digits-Dataset/Y.npy')
img_size= 64
plt.subplot(1,2,1)
plt.imshow(x_l[260].reshape(img_size,img_size))
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(x_l[900].reshape(img_size,img_size))
plt.axis('off')

(-0.5, 63.5, 63.5, -0.5)

In [15]:
X = np.concatenate((x_l[204:409], x_l[822:1027]),axis=0)
z= np.zeros(205)
o= np.ones(205)
Y = np.concatenate((z,o),axis=0).reshape(X.shape[0],1)

print(X.shape,Y.shape)

#The shape of the X is (410, 64, 64)
#410 means that we have 410 images (zero and one signs)
#64 means that our image size is 64x64 (64x64 pixels)
#The shape of the Y is (410,1)
#410 means that we have 410 labels (0 and 1)

(410, 64, 64) (410, 1)
